# Model Development and Fine tuning

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 755.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 61.1 MB/s eta 0:00:00


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.5 MB/s eta 0:00:00


In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
model = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    eos_token_id=tokenizer.eos_token_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
prompt = f"Translate this to korean: Hello"
generated = pipeline(prompt, max_length=25, num_return_sequences=1, truncation=True)
translation = generated[0]['generated_text']
print(translation)

Translate this to korean: Hello. I am a Korean teacher. I teach English. I teach Korean. I


In [ ]:
prompt = f"Translate to English: 안녕하세요"
generated = pipeline(prompt, max_length=25, num_return_sequences=1)
translation = generated[0]['generated_text']
print(translation)

Translate to English: 안녕하세요.
(South Korea) I'm fine,


In [ ]:
def translate_text(input_text):
    prompt = f"{input_text}"
    generated = pipeline(prompt, max_length=100, num_return_sequences=1)
    translation = generated[0]['generated_text']
    # Post-process the translation if needed
    return print(translation)

In [ ]:
translate_text("Translate to English: 안녕하세요")

Translate to English: 안녕하세요 annyeonghaseyo! Hello! How are you? 죄송합니다 joh-sung-ham-ni-da.
Translate to English: Sorry, I don't understand. 죄송합니다 joh-sung-ham-ni-da.
Translate to English: Thank you. 감사합


# Preparing dataset for further training and fine-tuning

In [ ]:
!pip install tensorflow_datasets

In [ ]:
# Set padding token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
import requests
import gzip
import os

# Download the dataset
url = 'https://object.pouta.csc.fi/OPUS-CCMatrix/v1/mono/ko.txt.gz'
response = requests.get(url)

# Save the file
with open('ko.txt.gz', 'wb') as f:
    f.write(response.content)

# Unzip the file
with gzip.open('ko.txt.gz', 'rb') as f_in:
    with open('ko.txt', 'wb') as f_out:
        f_out.write(f_in.read())

# Read a subset of the lines
subset_size = 1000  # Adjust the size as needed
ko_sentences = []
with open('ko.txt', 'r', encoding='utf-8') as f:
    for _ in range(subset_size):
        line = f.readline()
        if not line:
            break
        ko_sentences.append(line.strip())


In [ ]:
# Adding formality levels to the sentences (for demonstration, all set to 'polite')
en_sentences = ["Hello, how are you?", "Good morning, sir. How are you today?", "Greetings. I hope you are well.", "What's up?", "Good evening. How can I assist you today?"]
ko_sentences = ko_sentences[:5]  # Using a small subset for demonstration

class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, en_sentences, ko_sentences, max_length=128):
        self.tokenizer = tokenizer
        self.en_sentences = en_sentences
        self.ko_sentences = ko_sentences
        self.max_length = max_length

    def __len__(self):
        return len(self.en_sentences)

    def __getitem__(self, idx):
        en_text = f"[polite] {self.en_sentences[idx]}"
        ko_text = self.ko_sentences[idx]

        inputs = self.tokenizer(
            en_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        targets = self.tokenizer(
            ko_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        input_ids = inputs["input_ids"].squeeze()
        attention_mask = inputs["attention_mask"].squeeze()
        target_ids = targets["input_ids"].squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": target_ids
        }

# Create dataset
train_dataset = TranslationDataset(tokenizer, en_sentences, ko_sentences)


In [2]:
!pip install datasets
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

AttributeError: module 'pyarrow.lib' has no attribute 'ListViewType'

In [3]:
!pip install opustools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 43.7 MB/s eta 0:00:00


In [10]:
import os
from opustools import OpusRead, OpusGet

# Set the download directory
download_dir = 'opus_data'
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# Set parameters for opus_get
corpus_name = 'OpenSubtitles'
source_lang = 'en'
target_lang = 'ko'
output_file_en = os.path.join(download_dir, 'OpenSubtitles.en-ko.en')
output_file_ko = os.path.join(download_dir, 'OpenSubtitles.en-ko.ko')
max_sentences = 10000  # Set a smaller number of sentences to fit in Colab GPU

# Download the dataset
opus_getter = OpusGet(
    source=source_lang,
    target=target_lang,
    directory=corpus_name,
    preprocess='moses',
    download_dir=download_dir
)

corpora = [{
    'corpus': corpus_name,
    'release': 'latest',
    'version': 'v2018',  # Include the version key
    'preprocessing': 'moses',
    'src': source_lang,
    'trg': target_lang,
    'url': 'https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/moses/en-ko.txt.zip',
    'size': 45 * 1024 * 1024  # Size in bytes
}]

opus_getter.download(corpora=corpora, total_size=str(45 * 1024 * 1024))

# Read and process a fraction of the data
opus_reader = OpusRead(
    directory=corpus_name,
    source=source_lang,
    target=target_lang,
    preprocess='moses',
    write=[output_file_en, output_file_ko],
    maximum=max_sentences,
    download_dir=download_dir
)
opus_reader.printPairs()






opus_data/OpenSubtitles_latest_moses_en-ko.txt.zip ... 100% of 47 GB


FileNotFoundError: [Errno 2] No such file or directory: 'opus_data/opus_data/OpenSubtitles.en-ko.en'

In [20]:
!opus_read -d OpenSubtitles -s en -t ko -m 10000 -p raw -w OpenSubtitles.en-ko.en OpenSubtitles.en-ko.ko


Parsing file "./OpenSubtitles_latest_xml_en-ko.xml.gz" ... 0.0%
Traceback (most recent call last):
  File "/usr/local/bin/opus_read", line 138, in <module>
    OpusRead(**vars(args)).printPairs()
  File "/usr/local/lib/python3.10/dist-packages/opustools/opus_read.py", line 268, in printPairs
    src_doc = self.of_handler.open_sentence_file(src_doc_name, 'src')
  File "/usr/local/lib/python3.10/dist-packages/opustools/opus_file_handler.py", line 125, in open_sentence_file
    self.open_zipfiles()
  File "/usr/local/lib/python3.10/dist-packages/opustools/opus_file_handler.py", line 98, in open_zipfiles
    self.src_zip, self.trg_zip = self.open_specific_zips(
  File "/usr/local/lib/python3.10/dist-packages/opustools/opus_file_handler.py", line 83, in open_specific_zips
    src_zip = zipfile.ZipFile(src_zip_name, 'r')
  File "/usr/lib/python3.10/zipfile.py", line 1269, in __init__
    self._RealGetContents()
  File "/usr/lib/python3.10/zipfile.py", line 1336, in _RealGetContents
    raise

In [ ]:
!opus_read -d OpenSubtitles -s en -t ko -m 10000 -p moses -w OpenSubtitles.en-ko.en OpenSubtitles.en-ko.ko


The following files are available for downloading:

